In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.market_basket_dataset import MarketBasketDataset, BinaryEncodingTransform, RemoveItemsTransform
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

In [2]:
k = 128
tree_depth = 6
device = 'cuda'
dataset_path = r"/mnt/qnap/ekosman/Groceries_dataset.csv"

## Create the market basket dataset and use one-hot encoding for items

In [3]:
dataset = MarketBasketDataset(dataset_path=dataset_path)

In [4]:
model = AutoEncoder(dataset.n_items, 50, 4).train().to(device)
epochs = 500
lr = 5e-3
batch_size = 512
log_every = 5

In [5]:
dataset.transform = torchvision.transforms.Compose([
    RemoveItemsTransform(p=0.5),
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [6]:
model.train()
data_iter = torch.utils.data.DataLoader(dataset,
                                     batch_size=batch_size,
                                     shuffle=True,
                                     num_workers=1,
                                     pin_memory=True)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, 'min', verbose=True, factor=0.7, threshold=1e-4)
knn_crt = KNNLoss(k=k).to(device)
losses = []
alpha = 10/170
gamma = 2
for epoch in range(epochs):
    total_loss = 0
    for iteration, (batch, target) in enumerate(data_iter):
        batch = batch.to(device)
        target = target.to(device)
        outputs, iterm = model(batch, return_intermidiate=True)
        mse_loss = F.binary_cross_entropy_with_logits(outputs, target, reduction='none')
        mask = torch.ones_like(mse_loss)
        mask[target == 0] = alpha ** gamma
        mask[target == 1] = (1 - alpha) ** gamma
        mse_loss = (mse_loss * mask).sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(iterm)
            if torch.isinf(knn_loss):
                knn_loss = 0
        except ValueError:
            knn_loss = torch.tensor(0)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(data_iter)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | BCE Loss: {mse_loss.item()}")
    
    scheduler.step(total_loss / (iteration + 1))
    losses.append(total_loss / (iteration + 1))

Epoch 0 / 500 | iteration 0 / 30 | Total Loss: 8.180225372314453 | KNN Loss: 6.229950904846191 | BCE Loss: 1.9502747058868408
Epoch 0 / 500 | iteration 5 / 30 | Total Loss: 8.183635711669922 | KNN Loss: 6.229683876037598 | BCE Loss: 1.9539520740509033
Epoch 0 / 500 | iteration 10 / 30 | Total Loss: 8.158685684204102 | KNN Loss: 6.2299699783325195 | BCE Loss: 1.928715467453003
Epoch 0 / 500 | iteration 15 / 30 | Total Loss: 8.198995590209961 | KNN Loss: 6.229693412780762 | BCE Loss: 1.9693019390106201
Epoch 0 / 500 | iteration 20 / 30 | Total Loss: 8.157793998718262 | KNN Loss: 6.229772567749023 | BCE Loss: 1.9280214309692383
Epoch 0 / 500 | iteration 25 / 30 | Total Loss: 8.0819673538208 | KNN Loss: 6.229701042175293 | BCE Loss: 1.8522663116455078
Epoch 1 / 500 | iteration 0 / 30 | Total Loss: 8.16765022277832 | KNN Loss: 6.229369640350342 | BCE Loss: 1.9382810592651367
Epoch 1 / 500 | iteration 5 / 30 | Total Loss: 8.128780364990234 | KNN Loss: 6.229136943817139 | BCE Loss: 1.89964354

Epoch 10 / 500 | iteration 25 / 30 | Total Loss: 7.07936954498291 | KNN Loss: 5.899815082550049 | BCE Loss: 1.1795545816421509
Epoch 11 / 500 | iteration 0 / 30 | Total Loss: 7.034160614013672 | KNN Loss: 5.883945465087891 | BCE Loss: 1.1502151489257812
Epoch 11 / 500 | iteration 5 / 30 | Total Loss: 6.996936798095703 | KNN Loss: 5.832756519317627 | BCE Loss: 1.1641805171966553
Epoch 11 / 500 | iteration 10 / 30 | Total Loss: 6.9601149559021 | KNN Loss: 5.7869720458984375 | BCE Loss: 1.1731430292129517
Epoch 11 / 500 | iteration 15 / 30 | Total Loss: 6.9008636474609375 | KNN Loss: 5.745500564575195 | BCE Loss: 1.155362844467163
Epoch 11 / 500 | iteration 20 / 30 | Total Loss: 6.829466819763184 | KNN Loss: 5.708059787750244 | BCE Loss: 1.1214070320129395
Epoch 11 / 500 | iteration 25 / 30 | Total Loss: 6.806700706481934 | KNN Loss: 5.70042610168457 | BCE Loss: 1.1062748432159424
Epoch 12 / 500 | iteration 0 / 30 | Total Loss: 6.7193603515625 | KNN Loss: 5.615592002868652 | BCE Loss: 1.1

Epoch 21 / 500 | iteration 20 / 30 | Total Loss: 6.136683464050293 | KNN Loss: 5.08859395980835 | BCE Loss: 1.0480892658233643
Epoch 21 / 500 | iteration 25 / 30 | Total Loss: 6.154345512390137 | KNN Loss: 5.095767498016357 | BCE Loss: 1.0585782527923584
Epoch 22 / 500 | iteration 0 / 30 | Total Loss: 6.227887153625488 | KNN Loss: 5.146585464477539 | BCE Loss: 1.0813019275665283
Epoch 22 / 500 | iteration 5 / 30 | Total Loss: 6.109425067901611 | KNN Loss: 5.08043098449707 | BCE Loss: 1.028994083404541
Epoch 22 / 500 | iteration 10 / 30 | Total Loss: 6.174731731414795 | KNN Loss: 5.090341091156006 | BCE Loss: 1.084390640258789
Epoch 22 / 500 | iteration 15 / 30 | Total Loss: 6.132198333740234 | KNN Loss: 5.077462673187256 | BCE Loss: 1.054735779762268
Epoch 22 / 500 | iteration 20 / 30 | Total Loss: 6.128279209136963 | KNN Loss: 5.067727565765381 | BCE Loss: 1.060551643371582
Epoch 22 / 500 | iteration 25 / 30 | Total Loss: 6.169456958770752 | KNN Loss: 5.112816333770752 | BCE Loss: 1.0

Epoch 32 / 500 | iteration 15 / 30 | Total Loss: 6.130435943603516 | KNN Loss: 5.046717166900635 | BCE Loss: 1.0837187767028809
Epoch 32 / 500 | iteration 20 / 30 | Total Loss: 6.0779924392700195 | KNN Loss: 5.037525653839111 | BCE Loss: 1.0404667854309082
Epoch 32 / 500 | iteration 25 / 30 | Total Loss: 6.108156204223633 | KNN Loss: 5.0492377281188965 | BCE Loss: 1.0589187145233154
Epoch 33 / 500 | iteration 0 / 30 | Total Loss: 6.203396797180176 | KNN Loss: 5.093068599700928 | BCE Loss: 1.1103284358978271
Epoch 33 / 500 | iteration 5 / 30 | Total Loss: 6.112232208251953 | KNN Loss: 5.073187351226807 | BCE Loss: 1.0390450954437256
Epoch 33 / 500 | iteration 10 / 30 | Total Loss: 6.130401611328125 | KNN Loss: 5.056979179382324 | BCE Loss: 1.0734226703643799
Epoch 33 / 500 | iteration 15 / 30 | Total Loss: 6.106178283691406 | KNN Loss: 5.061697006225586 | BCE Loss: 1.0444811582565308
Epoch 33 / 500 | iteration 20 / 30 | Total Loss: 6.100916862487793 | KNN Loss: 5.063270568847656 | BCE L

Epoch 43 / 500 | iteration 10 / 30 | Total Loss: 6.0896077156066895 | KNN Loss: 5.037966728210449 | BCE Loss: 1.0516409873962402
Epoch 43 / 500 | iteration 15 / 30 | Total Loss: 6.083768844604492 | KNN Loss: 5.039661884307861 | BCE Loss: 1.04410719871521
Epoch 43 / 500 | iteration 20 / 30 | Total Loss: 6.115143775939941 | KNN Loss: 5.054600238800049 | BCE Loss: 1.0605437755584717
Epoch 43 / 500 | iteration 25 / 30 | Total Loss: 6.11208438873291 | KNN Loss: 5.058661937713623 | BCE Loss: 1.0534226894378662
Epoch 44 / 500 | iteration 0 / 30 | Total Loss: 6.074542999267578 | KNN Loss: 5.037545680999756 | BCE Loss: 1.0369970798492432
Epoch 44 / 500 | iteration 5 / 30 | Total Loss: 6.1285080909729 | KNN Loss: 5.04819917678833 | BCE Loss: 1.0803089141845703
Epoch 44 / 500 | iteration 10 / 30 | Total Loss: 6.118646621704102 | KNN Loss: 5.072786331176758 | BCE Loss: 1.0458600521087646
Epoch 44 / 500 | iteration 15 / 30 | Total Loss: 6.0933332443237305 | KNN Loss: 5.0483551025390625 | BCE Loss: 

...

Epoch 449 / 500 | iteration 15 / 30 | Total Loss: 5.998149394989014 | KNN Loss: 4.985795974731445 | BCE Loss: 1.0123534202575684
Epoch 449 / 500 | iteration 20 / 30 | Total Loss: 5.988480567932129 | KNN Loss: 4.9815239906311035 | BCE Loss: 1.0069568157196045
Epoch 449 / 500 | iteration 25 / 30 | Total Loss: 5.9784064292907715 | KNN Loss: 4.990254878997803 | BCE Loss: 0.9881516098976135
Epoch 450 / 500 | iteration 0 / 30 | Total Loss: 6.009145259857178 | KNN Loss: 4.9896345138549805 | BCE Loss: 1.0195107460021973
Epoch 450 / 500 | iteration 5 / 30 | Total Loss: 6.014015197753906 | KNN Loss: 5.000187873840332 | BCE Loss: 1.0138274431228638
Epoch 450 / 500 | iteration 10 / 30 | Total Loss: 6.029138565063477 | KNN Loss: 4.995615482330322 | BCE Loss: 1.0335230827331543
Epoch 450 / 500 | iteration 15 / 30 | Total Loss: 6.070472717285156 | KNN Loss: 5.054712772369385 | BCE Loss: 1.0157597064971924
Epoch 450 / 500 | iteration 20 / 30 | Total Loss: 6.025610446929932 | KNN Loss: 5.00410985946655

Epoch 460 / 500 | iteration 5 / 30 | Total Loss: 6.076190948486328 | KNN Loss: 5.036505699157715 | BCE Loss: 1.0396854877471924
Epoch 460 / 500 | iteration 10 / 30 | Total Loss: 5.98680305480957 | KNN Loss: 4.9798102378845215 | BCE Loss: 1.0069929361343384
Epoch 460 / 500 | iteration 15 / 30 | Total Loss: 6.058855056762695 | KNN Loss: 5.041995048522949 | BCE Loss: 1.016859769821167
Epoch 460 / 500 | iteration 20 / 30 | Total Loss: 6.000916481018066 | KNN Loss: 4.994308948516846 | BCE Loss: 1.0066072940826416
Epoch 460 / 500 | iteration 25 / 30 | Total Loss: 6.026651859283447 | KNN Loss: 5.011651992797852 | BCE Loss: 1.0149997472763062
Epoch 461 / 500 | iteration 0 / 30 | Total Loss: 6.0110392570495605 | KNN Loss: 4.977228164672852 | BCE Loss: 1.033811092376709
Epoch 461 / 500 | iteration 5 / 30 | Total Loss: 6.058467864990234 | KNN Loss: 5.020451545715332 | BCE Loss: 1.0380160808563232
Epoch 461 / 500 | iteration 10 / 30 | Total Loss: 6.073695182800293 | KNN Loss: 5.017346382141113 | B

Epoch 470 / 500 | iteration 25 / 30 | Total Loss: 6.022159099578857 | KNN Loss: 4.992762088775635 | BCE Loss: 1.0293970108032227
Epoch 471 / 500 | iteration 0 / 30 | Total Loss: 6.073005676269531 | KNN Loss: 5.009349822998047 | BCE Loss: 1.0636560916900635
Epoch 471 / 500 | iteration 5 / 30 | Total Loss: 6.052948951721191 | KNN Loss: 5.012661933898926 | BCE Loss: 1.0402872562408447
Epoch 471 / 500 | iteration 10 / 30 | Total Loss: 6.033650875091553 | KNN Loss: 5.002420425415039 | BCE Loss: 1.0312303304672241
Epoch 471 / 500 | iteration 15 / 30 | Total Loss: 6.046316623687744 | KNN Loss: 5.022641181945801 | BCE Loss: 1.0236754417419434
Epoch 471 / 500 | iteration 20 / 30 | Total Loss: 6.02301025390625 | KNN Loss: 5.001128196716309 | BCE Loss: 1.0218818187713623
Epoch 471 / 500 | iteration 25 / 30 | Total Loss: 6.0301618576049805 | KNN Loss: 5.008731365203857 | BCE Loss: 1.021430253982544
Epoch 472 / 500 | iteration 0 / 30 | Total Loss: 6.019319534301758 | KNN Loss: 5.002521991729736 | B

Epoch 481 / 500 | iteration 15 / 30 | Total Loss: 6.055817127227783 | KNN Loss: 5.0468010902404785 | BCE Loss: 1.0090160369873047
Epoch 481 / 500 | iteration 20 / 30 | Total Loss: 6.0202507972717285 | KNN Loss: 4.999390125274658 | BCE Loss: 1.0208606719970703
Epoch 481 / 500 | iteration 25 / 30 | Total Loss: 6.018528461456299 | KNN Loss: 5.000139236450195 | BCE Loss: 1.018389344215393
Epoch 482 / 500 | iteration 0 / 30 | Total Loss: 6.071181774139404 | KNN Loss: 5.0631022453308105 | BCE Loss: 1.0080796480178833
Epoch 482 / 500 | iteration 5 / 30 | Total Loss: 6.069654941558838 | KNN Loss: 5.0339765548706055 | BCE Loss: 1.0356782674789429
Epoch 482 / 500 | iteration 10 / 30 | Total Loss: 6.013039588928223 | KNN Loss: 5.013042449951172 | BCE Loss: 0.9999969005584717
Epoch 482 / 500 | iteration 15 / 30 | Total Loss: 6.068719863891602 | KNN Loss: 5.020889759063721 | BCE Loss: 1.0478301048278809
Epoch 482 / 500 | iteration 20 / 30 | Total Loss: 6.03743839263916 | KNN Loss: 4.987823009490967

Epoch 492 / 500 | iteration 5 / 30 | Total Loss: 6.004672527313232 | KNN Loss: 5.000941276550293 | BCE Loss: 1.003731369972229
Epoch 492 / 500 | iteration 10 / 30 | Total Loss: 6.00396728515625 | KNN Loss: 4.997598648071289 | BCE Loss: 1.00636887550354
Epoch 492 / 500 | iteration 15 / 30 | Total Loss: 6.017963886260986 | KNN Loss: 5.013737201690674 | BCE Loss: 1.004226803779602
Epoch 492 / 500 | iteration 20 / 30 | Total Loss: 6.02263069152832 | KNN Loss: 5.001904487609863 | BCE Loss: 1.020725965499878
Epoch 492 / 500 | iteration 25 / 30 | Total Loss: 6.031940460205078 | KNN Loss: 5.015169143676758 | BCE Loss: 1.0167710781097412
Epoch 493 / 500 | iteration 0 / 30 | Total Loss: 6.021288871765137 | KNN Loss: 4.998803615570068 | BCE Loss: 1.0224854946136475
Epoch 493 / 500 | iteration 5 / 30 | Total Loss: 6.0339555740356445 | KNN Loss: 4.987903118133545 | BCE Loss: 1.0460525751113892
Epoch 493 / 500 | iteration 10 / 30 | Total Loss: 6.079273223876953 | KNN Loss: 5.01887845993042 | BCE Los

In [7]:
outputs, iterm = model(dataset[67][0].unsqueeze(0).to(device), return_intermidiate=True)
print(outputs)
print(outputs.min())
print(outputs.max())

tensor([[ 2.2265,  3.8615,  2.5736,  2.9090,  2.8241,  0.6991,  2.6035,  1.6462,
          2.3869,  2.0602,  2.1964,  2.0600,  0.6111,  1.8932,  1.3220,  1.4583,
          2.9208,  2.6505,  2.8708,  2.3284,  1.7294,  3.1018,  2.3613,  2.6003,
          1.9957,  1.8294,  2.0026,  1.2849,  1.4417,  0.4059, -0.1320,  1.0266,
          0.2938,  0.9174,  1.6701,  1.4917,  1.0858,  3.2999,  0.7243,  1.4092,
          0.8491, -0.9957, -0.2098,  2.2923,  2.1144,  0.7912, -0.1759,  0.0579,
          1.4154,  1.9824,  1.9156,  0.2031,  1.2618,  0.4512, -0.5825,  1.0971,
          1.3924,  1.4737,  1.2272,  1.9045,  0.5839,  0.9296,  0.1225,  1.7981,
          1.3353,  1.7348, -1.8844,  0.4145,  2.3761,  2.0298,  2.5776,  0.4516,
          1.4844,  2.5514,  1.8596,  1.3033,  0.3155,  0.7763,  0.1432,  1.6417,
          0.0722,  0.5226,  1.9932, -0.3415,  0.2397, -1.0558, -2.4648, -0.2253,
          0.5830, -1.8698,  0.5544, -0.1350, -0.5117, -0.9409,  0.6519,  1.3184,
         -0.6455, -0.7293,  

In [8]:
plt.figure()
plt.plot(losses)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
dataset.transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [10]:
dataset_ = [d[0].to('cpu') for d in dataset]

In [11]:
model = model.eval().to('cpu')
projections = model.calculate_intermidiate(dataset_)

100%|██████████| 15/15 [00:00<00:00, 88.94it/s]


In [12]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit DBSCAN and calculate indices


In [13]:
clusters = DBSCAN(eps=0.1, min_samples=80).fit_predict(projections)
# scores = []
# best_score = float('inf')
# clusters = None
# range_ = list(range(5, 20))
# for k in tqdm(range_):
#     y = GaussianMixture(n_components=k).fit_predict(projections)
#     cur_score = davies_bouldin_score(projections, y)
#     scores.append(cur_score)
    
#     if cur_score < best_score:
#         best_score = cur_score
#         clusters = y

In [14]:
perplexity = 100

p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn import tree
# from sklearn.tree import _tree

In [16]:
# tensor_dataset = torch.stack(dataset_)

In [17]:
# clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=5)
# clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
# print(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
# print(clf.get_depth())

In [18]:
# scores = []
# for min_samples in range(1,50, 1):
#     clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=min_samples)
#     clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
#     scores.append(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
    
# plt.figure()
# plt.plot(list(range(1,50, 1)), scores)
# plt.show()

In [19]:
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
#             p1 += [f"({name} <= {np.round(threshold, 3)})"]
            p1 += [(name, '<=', np.round(threshold, 3))]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [(name, '>', np.round(threshold, 3))]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = []
        
        for p in path[:-1]:
            rule += [p]
        target = " then "
        if class_names is None:
            target += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            target += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
           
        proba = np.round(100.0*classes[l]/np.sum(classes),2)
        target += f" | based on {path[-1][1]:,} samples"
        rule_wrapper = {'target': target, 'rule': rule, 'proba': proba}
        rules += [rule_wrapper]
        
    return rules

In [20]:
# rules = get_rules(clf, dataset.items, clusters[clusters != -1])

# for rule in rules:
#     n_pos = 0
#     for c,p,v in rule['rule']:
#         if p == '>':
#             n_pos += 1
#     rule['pos'] = n_pos

In [21]:
# plt.figure()
# probs = [r['proba'] for r in rules]
# plt.hist(probs, bins = 100)
# plt.show()

In [22]:
# rules = sorted(rules, key=lambda x:x['pos'])
# rules = [r for r in rules if r['proba'] > 50]
# print(len(rules))

In [23]:
# for i in range(17):
#     r_i = rules[i]
#     print(f"------------- rule {i} length {len(r_i)} -------------")
#     print(r_i)

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [24]:
tensor_dataset = torch.stack(dataset_)

In [25]:
tree_dataset = list(zip(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [26]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [27]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [28]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 100
output_dim = len(set(clusters))
log_interval = 1
use_cuda = device != 'cpu'

In [29]:
tree = SDT(input_dim=tensor_dataset.shape[1], output_dim=len(clusters - 1), depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [30]:
losses = []
accs = []
sparsity = []

In [31]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
Epoch: 00 | Batch: 000 / 029 | Total loss: 9.575 | Reg loss: 0.007 | Tree loss: 9.575 | Accuracy: 0.000000 | 0.098 sec/iter
Epoch: 00 | Batch: 001 / 029 | Total loss: 9.570 | Reg loss: 0.007 | Tree loss: 9.570 | Accuracy: 0.000000 | 0.086 sec/iter
Epoch: 00 | Batch: 002 / 029 | Total loss: 9.548 | Reg loss: 0.007 | Tree loss: 9.548 | Accuracy: 0.000000 | 0.079 sec/iter
Epoch: 00 | Batch: 003 / 029 | Total loss: 9.531 | Reg loss: 0.007 | Tree loss: 9.531 | Accuracy: 0.000000 | 0.076 sec/iter
Epoch: 00 | Batch: 004 / 029 | Total loss: 9.536 | Reg loss: 0.007 | Tree loss: 9.536 | Accuracy: 0.000000 | 0.074 sec/iter
Epoch: 00 | Batch: 005 / 029 | Total loss: 9.522 | Reg loss: 0.007 | Tree loss: 9.522 | Accuracy: 0.000000 | 0.073 sec/iter
Epoch: 00 | Batch: 006 / 029 | Total loss: 9.507 | Reg loss: 0.007 | Tree loss: 9.507 | Accuracy: 0.000000 | 0.072 sec/iter
Epoch: 00 | Batch: 007 / 029 | Total loss: 

Epoch: 02 | Batch: 007 / 029 | Total loss: 9.179 | Reg loss: 0.007 | Tree loss: 9.179 | Accuracy: 0.238281 | 0.065 sec/iter
Epoch: 02 | Batch: 008 / 029 | Total loss: 9.143 | Reg loss: 0.007 | Tree loss: 9.143 | Accuracy: 0.312500 | 0.065 sec/iter
Epoch: 02 | Batch: 009 / 029 | Total loss: 9.134 | Reg loss: 0.007 | Tree loss: 9.134 | Accuracy: 0.271484 | 0.065 sec/iter
Epoch: 02 | Batch: 010 / 029 | Total loss: 9.140 | Reg loss: 0.008 | Tree loss: 9.140 | Accuracy: 0.259766 | 0.065 sec/iter
Epoch: 02 | Batch: 011 / 029 | Total loss: 9.093 | Reg loss: 0.008 | Tree loss: 9.093 | Accuracy: 0.355469 | 0.065 sec/iter
Epoch: 02 | Batch: 012 / 029 | Total loss: 9.109 | Reg loss: 0.008 | Tree loss: 9.109 | Accuracy: 0.279297 | 0.065 sec/iter
Epoch: 02 | Batch: 013 / 029 | Total loss: 9.074 | Reg loss: 0.008 | Tree loss: 9.074 | Accuracy: 0.304688 | 0.065 sec/iter
Epoch: 02 | Batch: 014 / 029 | Total loss: 9.088 | Reg loss: 0.009 | Tree loss: 9.088 | Accuracy: 0.285156 | 0.065 sec/iter
Epoch: 0

Epoch: 04 | Batch: 013 / 029 | Total loss: 8.747 | Reg loss: 0.012 | Tree loss: 8.747 | Accuracy: 0.314453 | 0.065 sec/iter
Epoch: 04 | Batch: 014 / 029 | Total loss: 8.735 | Reg loss: 0.012 | Tree loss: 8.735 | Accuracy: 0.304688 | 0.065 sec/iter
Epoch: 04 | Batch: 015 / 029 | Total loss: 8.733 | Reg loss: 0.013 | Tree loss: 8.733 | Accuracy: 0.318359 | 0.065 sec/iter
Epoch: 04 | Batch: 016 / 029 | Total loss: 8.729 | Reg loss: 0.013 | Tree loss: 8.729 | Accuracy: 0.294922 | 0.065 sec/iter
Epoch: 04 | Batch: 017 / 029 | Total loss: 8.744 | Reg loss: 0.013 | Tree loss: 8.744 | Accuracy: 0.250000 | 0.065 sec/iter
Epoch: 04 | Batch: 018 / 029 | Total loss: 8.720 | Reg loss: 0.013 | Tree loss: 8.720 | Accuracy: 0.273438 | 0.065 sec/iter
Epoch: 04 | Batch: 019 / 029 | Total loss: 8.696 | Reg loss: 0.014 | Tree loss: 8.696 | Accuracy: 0.285156 | 0.065 sec/iter
Epoch: 04 | Batch: 020 / 029 | Total loss: 8.695 | Reg loss: 0.014 | Tree loss: 8.695 | Accuracy: 0.263672 | 0.065 sec/iter
Epoch: 0

Epoch: 06 | Batch: 020 / 029 | Total loss: 8.309 | Reg loss: 0.018 | Tree loss: 8.309 | Accuracy: 0.283203 | 0.065 sec/iter
Epoch: 06 | Batch: 021 / 029 | Total loss: 8.285 | Reg loss: 0.019 | Tree loss: 8.285 | Accuracy: 0.289062 | 0.065 sec/iter
Epoch: 06 | Batch: 022 / 029 | Total loss: 8.269 | Reg loss: 0.019 | Tree loss: 8.269 | Accuracy: 0.304688 | 0.065 sec/iter
Epoch: 06 | Batch: 023 / 029 | Total loss: 8.266 | Reg loss: 0.019 | Tree loss: 8.266 | Accuracy: 0.281250 | 0.065 sec/iter
Epoch: 06 | Batch: 024 / 029 | Total loss: 8.240 | Reg loss: 0.020 | Tree loss: 8.240 | Accuracy: 0.296875 | 0.065 sec/iter
Epoch: 06 | Batch: 025 / 029 | Total loss: 8.235 | Reg loss: 0.020 | Tree loss: 8.235 | Accuracy: 0.289062 | 0.065 sec/iter
Epoch: 06 | Batch: 026 / 029 | Total loss: 8.226 | Reg loss: 0.020 | Tree loss: 8.226 | Accuracy: 0.289062 | 0.065 sec/iter
Epoch: 06 | Batch: 027 / 029 | Total loss: 8.193 | Reg loss: 0.021 | Tree loss: 8.193 | Accuracy: 0.296875 | 0.065 sec/iter
Epoch: 0

Average sparseness: 0.9821428571428567
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
Epoch: 09 | Batch: 000 / 029 | Total loss: 8.103 | Reg loss: 0.019 | Tree loss: 8.103 | Accuracy: 0.267578 | 0.065 sec/iter
Epoch: 09 | Batch: 001 / 029 | Total loss: 8.030 | Reg loss: 0.019 | Tree loss: 8.030 | Accuracy: 0.310547 | 0.065 sec/iter
Epoch: 09 | Batch: 002 / 029 | Total loss: 8.023 | Reg loss: 0.019 | Tree loss: 8.023 | Accuracy: 0.283203 | 0.065 sec/iter
Epoch: 09 | Batch: 003 / 029 | Total loss: 8.029 | Reg loss: 0.020 | Tree loss: 8.029 | Accuracy: 0.287109 | 0.065 sec/iter
Epoch: 09 | Batch: 004 / 029 | Total loss: 7.997 | Reg loss: 0.020 | Tree loss: 7.997 | Accuracy: 0.281250 | 0.065 sec/iter
Epoch: 09 | Batch: 005 / 029 | Total loss: 7.975 | Reg loss: 0.020 | Tree loss: 7.975 | Accuracy: 0.300781 | 0.065 sec/iter
Epoch: 09 | Batch: 006 / 029 | Total loss: 7.964 | Reg loss: 0.020 | Tree los

...

Epoch: 89 | Batch: 011 / 029 | Total loss: 1.959 | Reg loss: 0.036 | Tree loss: 1.959 | Accuracy: 0.322266 | 0.069 sec/iter
Epoch: 89 | Batch: 012 / 029 | Total loss: 1.968 | Reg loss: 0.036 | Tree loss: 1.968 | Accuracy: 0.322266 | 0.069 sec/iter
Epoch: 89 | Batch: 013 / 029 | Total loss: 1.996 | Reg loss: 0.036 | Tree loss: 1.996 | Accuracy: 0.259766 | 0.069 sec/iter
Epoch: 89 | Batch: 014 / 029 | Total loss: 1.982 | Reg loss: 0.036 | Tree loss: 1.982 | Accuracy: 0.255859 | 0.069 sec/iter
Epoch: 89 | Batch: 015 / 029 | Total loss: 1.949 | Reg loss: 0.036 | Tree loss: 1.949 | Accuracy: 0.279297 | 0.069 sec/iter
Epoch: 89 | Batch: 016 / 029 | Total loss: 1.917 | Reg loss: 0.036 | Tree loss: 1.917 | Accuracy: 0.296875 | 0.069 sec/iter
Epoch: 89 | Batch: 017 / 029 | Total loss: 1.956 | Reg loss: 0.036 | Tree loss: 1.956 | Accuracy: 0.283203 | 0.069 sec/iter
Epoch: 89 | Batch: 018 / 029 | Total loss: 1.957 | Reg loss: 0.036 | Tree loss: 1.957 | Accuracy: 0.283203 | 0.069 sec/iter
Epoch: 8

Epoch: 91 | Batch: 019 / 029 | Total loss: 1.930 | Reg loss: 0.036 | Tree loss: 1.930 | Accuracy: 0.306641 | 0.069 sec/iter
Epoch: 91 | Batch: 020 / 029 | Total loss: 1.907 | Reg loss: 0.036 | Tree loss: 1.907 | Accuracy: 0.292969 | 0.069 sec/iter
Epoch: 91 | Batch: 021 / 029 | Total loss: 1.961 | Reg loss: 0.036 | Tree loss: 1.961 | Accuracy: 0.289062 | 0.069 sec/iter
Epoch: 91 | Batch: 022 / 029 | Total loss: 1.885 | Reg loss: 0.036 | Tree loss: 1.885 | Accuracy: 0.281250 | 0.069 sec/iter
Epoch: 91 | Batch: 023 / 029 | Total loss: 1.854 | Reg loss: 0.036 | Tree loss: 1.854 | Accuracy: 0.292969 | 0.069 sec/iter
Epoch: 91 | Batch: 024 / 029 | Total loss: 1.905 | Reg loss: 0.037 | Tree loss: 1.905 | Accuracy: 0.291016 | 0.069 sec/iter
Epoch: 91 | Batch: 025 / 029 | Total loss: 1.935 | Reg loss: 0.037 | Tree loss: 1.935 | Accuracy: 0.277344 | 0.069 sec/iter
Epoch: 91 | Batch: 026 / 029 | Total loss: 1.787 | Reg loss: 0.037 | Tree loss: 1.787 | Accuracy: 0.277344 | 0.069 sec/iter
Epoch: 9

Epoch: 93 | Batch: 026 / 029 | Total loss: 1.811 | Reg loss: 0.037 | Tree loss: 1.811 | Accuracy: 0.300781 | 0.069 sec/iter
Epoch: 93 | Batch: 027 / 029 | Total loss: 1.893 | Reg loss: 0.037 | Tree loss: 1.893 | Accuracy: 0.287109 | 0.069 sec/iter
Epoch: 93 | Batch: 028 / 029 | Total loss: 1.895 | Reg loss: 0.037 | Tree loss: 1.895 | Accuracy: 0.253247 | 0.069 sec/iter
Average sparseness: 0.9821428571428567
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
Epoch: 94 | Batch: 000 / 029 | Total loss: 2.163 | Reg loss: 0.035 | Tree loss: 2.163 | Accuracy: 0.271484 | 0.069 sec/iter
Epoch: 94 | Batch: 001 / 029 | Total loss: 2.150 | Reg loss: 0.035 | Tree loss: 2.150 | Accuracy: 0.308594 | 0.069 sec/iter
Epoch: 94 | Batch: 002 / 029 | Total loss: 2.127 | Reg loss: 0.035 | Tree loss: 2.127 | Accuracy: 0.294922 | 0.069 sec/iter
Epoch: 94 | Batch: 003 / 029 | Total loss: 2.119 | Reg loss: 0.035 | Tree los

Epoch: 96 | Batch: 003 / 029 | Total loss: 2.133 | Reg loss: 0.035 | Tree loss: 2.133 | Accuracy: 0.281250 | 0.069 sec/iter
Epoch: 96 | Batch: 004 / 029 | Total loss: 2.096 | Reg loss: 0.035 | Tree loss: 2.096 | Accuracy: 0.267578 | 0.069 sec/iter
Epoch: 96 | Batch: 005 / 029 | Total loss: 2.115 | Reg loss: 0.035 | Tree loss: 2.115 | Accuracy: 0.257812 | 0.069 sec/iter
Epoch: 96 | Batch: 006 / 029 | Total loss: 2.028 | Reg loss: 0.035 | Tree loss: 2.028 | Accuracy: 0.302734 | 0.069 sec/iter
Epoch: 96 | Batch: 007 / 029 | Total loss: 2.031 | Reg loss: 0.035 | Tree loss: 2.031 | Accuracy: 0.261719 | 0.069 sec/iter
Epoch: 96 | Batch: 008 / 029 | Total loss: 2.025 | Reg loss: 0.035 | Tree loss: 2.025 | Accuracy: 0.320312 | 0.069 sec/iter
Epoch: 96 | Batch: 009 / 029 | Total loss: 2.035 | Reg loss: 0.035 | Tree loss: 2.035 | Accuracy: 0.281250 | 0.069 sec/iter
Epoch: 96 | Batch: 010 / 029 | Total loss: 2.009 | Reg loss: 0.035 | Tree loss: 2.009 | Accuracy: 0.294922 | 0.069 sec/iter
Epoch: 9

Epoch: 98 | Batch: 010 / 029 | Total loss: 2.010 | Reg loss: 0.035 | Tree loss: 2.010 | Accuracy: 0.308594 | 0.069 sec/iter
Epoch: 98 | Batch: 011 / 029 | Total loss: 2.027 | Reg loss: 0.035 | Tree loss: 2.027 | Accuracy: 0.306641 | 0.069 sec/iter
Epoch: 98 | Batch: 012 / 029 | Total loss: 2.059 | Reg loss: 0.035 | Tree loss: 2.059 | Accuracy: 0.289062 | 0.069 sec/iter
Epoch: 98 | Batch: 013 / 029 | Total loss: 1.990 | Reg loss: 0.035 | Tree loss: 1.990 | Accuracy: 0.289062 | 0.069 sec/iter
Epoch: 98 | Batch: 014 / 029 | Total loss: 1.944 | Reg loss: 0.035 | Tree loss: 1.944 | Accuracy: 0.281250 | 0.069 sec/iter
Epoch: 98 | Batch: 015 / 029 | Total loss: 1.937 | Reg loss: 0.036 | Tree loss: 1.937 | Accuracy: 0.285156 | 0.069 sec/iter
Epoch: 98 | Batch: 016 / 029 | Total loss: 1.861 | Reg loss: 0.036 | Tree loss: 1.861 | Accuracy: 0.296875 | 0.069 sec/iter
Epoch: 98 | Batch: 017 / 029 | Total loss: 1.926 | Reg loss: 0.036 | Tree loss: 1.926 | Accuracy: 0.265625 | 0.069 sec/iter
Epoch: 9

In [32]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Tree Visualization

In [34]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Average height: 5.185185185185185


# Extract Rules

# Accumulate samples in the leaves

In [35]:
print(f"Number of patterns: {len(root.get_leaves())}")

Number of patterns: 27


In [36]:
method = 'greedy'

In [37]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

/home/eitan.k/.local/lib/python3.6/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


# Tighten boundaries

In [38]:
attr_names = dataset.items

# print(attr_names)
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")

7804
============== Pattern 1 ==============
============== Pattern 2 ==============
1659
============== Pattern 3 ==============
============== Pattern 4 ==============
============== Pattern 5 ==============
2261
============== Pattern 6 ==============
============== Pattern 7 ==============
345
============== Pattern 8 ==============
============== Pattern 9 ==============
============== Pattern 10 ==============
============== Pattern 11 ==============
============== Pattern 12 ==============
============== Pattern 13 ==============
2098
============== Pattern 14 ==============
============== Pattern 15 ==============
============== Pattern 16 ==============
============== Pattern 17 ==============
============== Pattern 18 ==============
============== Pattern 19 ==============
============== Pattern 20 ==============
============== Pattern 21 ==============
============== Pattern 22 ==============
56
============== Pattern 23 ==============
267
============== Pattern 24 =========

/home/eitan.k/EntangledExplainableClustering/soft_decision_tree/sdt_model.py:14: RuntimeWarning: divide by zero encountered in log
  return np.log(1 / (1 - x))
